# Meraki Python SDK Demo: Uplink Preference Restore

*This notebook demonstrates using the Meraki Python SDK to restore Internet (WAN) and VPN traffic uplink preferences, as well as custom performance classes, from an Excel file. If you have hundreds of WAN/VPN uplink preferences, they can be a challenge to manipulate. This demo seeks to prove how using the Meraki API and Python SDK can substantially streamline such complex deployments.*

If you haven't already, please consult the corresponding **Meraki Python SDK Demo: Uplink Preference Backup**.

If an admin has backed up his Internet and VPN traffic uplink preferences and custom performance classes, this tool will restore them to the Dashboard from the Excel file backup. This is a more advanced demo, intended for intermediate to advanced Python programmers, but has been documented thoroughly with the intention that even a determined Python beginner can understand the concepts involved.

If an admin can use the appropriate template Excel file and update it with the appropriate info, e.g. subnets, ports, and WAN preference, then this tool can push those preferences to the Dashboard for the desired network's MX appliance. With the Meraki Dashboard API, its SDK and Python, we can restore hundreds of preferences without using the GUI.

---

>NB: Throughout this notebook, we will print values for demonstration purposes. In a production Python script, the coder would likely remove these print statements to clean up the console output.

In this first cell, we import the required `meraki` and `os` modules, and open the Dashboard API connection using the SDK. We also import `openpyxl` for working with Excel files, and `netaddr` for working with IP addresses.

In [ ]:
# Install the relevant modules. If you are using a local editor (e.g. VS Code, rather than Colab) you can run these commands, without the preceding %, via a terminal. NB: Run `pip install meraki==` to find the latest version of the Meraki SDK. Uncomment these lines if you're using Google Colab.
#%pip install meraki
#%pip install openpyxl

# If you are using Google Colab, please ensure you have set up your environment variables as linked above, then delete the two lines of ''' to activate the following code:
'''
%pip install colab-env -qU
import colab_env
'''

# The Meraki SDK
import meraki
# The built-in OS module, to read environment variables
import os
# We're also going to import Python's built-in JSON module, but only to make the console output pretty. In production, you wouldn't need any of the printing calls at all, nor this import!
import json

# The openpyxl module, to manipulate Excel files
import openpyxl
# The datetime module, to generate timestamps
import datetime

# Treat your API key like a password. Store it in your environment variables as 'MERAKI_DASHBOARD_API_KEY' and let the SDK call it for you.
# Or, call it manually after importing Python's os module:
# API_KEY = os.getenv('MERAKI_DASHBOARD_API_KEY')

# Initialize the Dashboard connection.
dashboard = meraki.DashboardAPI(suppress_logging=True)

# We'll also create a few reusable strings for later interactivity.
string_constants = dict()
string_constants['CONFIRM'] = 'OK, are you sure you want to do this? This script does not have an "undo" feature.'
string_constants['CANCEL'] = 'OK. Operation canceled.'
string_constants['WORKING'] = 'Working...'
string_constants['COMPLETE'] = 'Operation complete.'
string_constants['NETWORK_SELECTED'] = 'Network selected.'
string_constants['NO_VALID_OPTIONS'] = 'There are no valid options. Please try again with an API key that has access to the appropriate resources.'

# Some of the parameters we'll work with are optional. This string defines what value will be put into a cell corresponding with a parameter that is not set on that rule.
string_constants['NOT_APPLICABLE'] = 'N/A'

# This script is interactive; user choices and data will be stored here.
user_choices = dict()
user_data = dict()

# Set the filename to use for the backup workbook
WORKBOOK_FILENAME = 'exampleBackups/downloaded_rules_workbook_2020-12-01 backup with wan and vpn uplink prefs and cpcs.xlsx'

A basic pretty print formatter, `printj()`. It will make reading JSON on the console easier, but won't be necessary in production scripts.

In [ ]:
def printj(ugly_json_object):
	
	# The json.dumps() method converts a JSON object into human-friendly formatted text
	pretty_json_string = json.dumps(ugly_json_object, indent = 2, sort_keys = False)

	return print(pretty_json_string)

We'll reuse the custom class we created in the backup script.

In [ ]:
class UserChoice:

    'A re-usable CLI option prompt.'

    def __init__(self, options_list=[], subject_of_choice='available options', single_option_noun='option', id_parameter='id', name_parameter='name', action_verb='choose', no_valid_options_message='no valid options'):

        self.options_list = options_list # options_list is a list of dictionaries containing attributes id_parameter and name_parameter  
        self.subject_of_choice = subject_of_choice # subject_of_choice is a string that names the subject of the user's choice. It is typically a plural noun.
        self.single_option_noun = single_option_noun # single_option_noun is a string that is a singular noun corresponding to the subject_of_choice
        self.id_parameter = id_parameter # id_parameter is a string that represents the name of the sub-parameter that serves as the ID value for the option in options_list. It should be a unique value for usability.
        self.name_parameter = name_parameter # name_paraemter is a string that represents the name of the sub-parameter that serves as the name value for the option in options_list. It does not need to be unique.
        self.action_verb = action_verb # action_verb is a string that represents the verb of the user's action. For example, to "choose"
        self.no_valid_options_message = no_valid_options_message # no_valid_options_message is a string that represents an error message if options_list is empty
              
        # Confirm there are options in the list
        if len(self.options_list):
            print(f'We found {len(self.options_list)} {self.subject_of_choice}:')

            # Label each option and show the user their choices.
            option_index = 0

            for option in self.options_list:
                print(f"{option_index}. {option[self.id_parameter]} with name {option[self.name_parameter]}")
                option_index+=1

            print(f'Which {self.single_option_noun} would you like to {self.action_verb}?')
            self.active_option = int(input(f'Choose 0-{option_index-1}:'))
            
            # Ask until the user provides valid input.
            while self.active_option not in list(range(option_index)):
                print(f'{self.active_option} is not a valid choice. Which {self.single_option_noun} would you like to {self.action_verb}?')
                self.active_option = int(input(f'Choose 0-{option_index-1}:'))

            print(f'Your {self.single_option_noun} is {self.options_list[self.active_option][self.name_parameter]}.')

            # Assign the class id and name vars to the chosen item's
            self.id = self.options_list[self.active_option][self.id_parameter]
            self.name = self.options_list[self.active_option][self.name_parameter]

## Pulling organization and network IDs

Most API calls require passing values for the organization ID and/or the network ID. Remember that `UserChoice` class we created earlier? We'll call that and supply parameters defining what the user can choose. Notice how, having defined the class earlier, we can re-use it with only a single declaration.

In [ ]:
# getOrganizations will return all orgs to which the supplied API key has access
user_choices['all_organizations'] = dashboard.organizations.getOrganizations()

# Prompt the user to pick an organization.
user_choices['organization'] = UserChoice(
    options_list=user_choices['all_organizations'], 
    subject_of_choice='organizations', 
    single_option_noun='organization', 
    no_valid_options_message=string_constants['NO_VALID_OPTIONS']
    )

## Identify networks with MX appliances, and prompt the user to choose one

We want to:

> Restore a backup of the uplink selection preferences, including custom performance classes.

We can only run this on networks that have appliance devices, so we will find networks where `productTypes` contains `appliance`. Then we'll ask the user to pick one, then pull the uplink selection rules from it.

Then let's ask the user which network they'd like to use.

In [ ]:
user_choices['all_networks'] = dashboard.organizations.getOrganizationNetworks(organizationId=user_choices['organization'].id)

# Find the networks with appliances
user_choices['networks_with_appliances']= [network for network in user_choices['all_networks'] if 'appliance' in network['productTypes']]

# If any are found, let the user choose a network. Otherwise, let the user know that none were found. The logic for this class is defined in a cell above.
user_choices['network_choice'] = UserChoice(
	options_list = user_choices['networks_with_appliances'], 
	subject_of_choice = 'networks with appliances', 
	single_option_noun = 'network'
	)

## Overall restore workflow

### Logical summary

The restore workflow summarized is:

1. Open a chosen Excel workbook that contains the backup information.
2. Parse each worksheet into a Python object structured according to the API documentation.
3. Restore the custom performance classes from the backup.
4. Restore the WAN (Internet) and VPN uplink preferences from the backup.

### Code summary

To structure the code, we'll break down the total functionality into discrete functions. These functions are where we define the operational logic for the restore. Most, if not all functions will return relevant information to be used by the next function in the restore procedure.

1. The first function will ingest the Excel spreadsheet with the backup information and return the data as a Python object, structured according to the API documentation.
2. Another function will restore the custom performance classes from the backup. This is a tricky operation for reasons you'll see below, but fully possible via Python methods and the Meraki SDK.
3. Another function will, if necessary, update the loaded backup object with new ID assignments, in case restoring the custom performance classes backup resulted in new class IDs.
4. Another function will restore the VPN preferences.
5. Another function will restore the WAN preferencess.
6. After definining each of those functions, we'll run them in succession to finalize the backup restoration.

Once you understand the fundamentals, consider how you might improve this script, either with additional functionality or UX improvements!

> NB: *Function* and *method* are used interchangeably. Python is an object-oriented language, and *method* is often preferred when discussing object-oriented programming.

In [ ]:
# Ingest an Excel spreadsheet with the appropriate worksheets, and create an object that can be pushed as a configuration API call

def load_uplink_prefs_workbook(workbook_filename):

	# Create a workbook object out of the actual workbook file
	loaded_workbook = openpyxl.load_workbook(workbook_filename, read_only=True)

	# Create empty rule lists to which we can add the rules defined in the workbook
	loaded_custom_performance_classes = []
	loaded_wan_uplink_prefs = []
	loaded_vpn_uplink_prefs = []

	# Open the worksheets
	loaded_custom_performance_classes_worksheet = loaded_workbook['customPerformanceClasses']
	loaded_wan_prefs_worksheet = loaded_workbook['wanUplinkPreferences']
	loaded_vpn_prefs_worksheet = loaded_workbook['vpnUplinkPreferences']

	## CUSTOM PERFORMANCE CLASSES ##
	# We'll also count the number of classes to help the user know that it's working.
	performance_class_count = 0	

	# For reference, the expected column order is
	# ID [0], Name [1], Max Latency [2], Max Jitter [3], Max Loss Percentage [4]

	# Append each performance class loaded_custom_performance_classes
	for row in loaded_custom_performance_classes_worksheet.iter_rows(min_row=2):
		
		# Let's start with an empty rule dictionary to which we'll add the relevant parameters
		performance_class = {}

		# Append the values
		performance_class['customPerformanceClassId'] = row[0].value
		performance_class['name'] = row[1].value
		performance_class['maxLatency'] = row[2].value
		performance_class['maxJitter'] = row[3].value
		performance_class['maxLossPercentage'] = row[4].value

		# Append the performance class to the loaded_custom_performance_classes list
		loaded_custom_performance_classes.append(performance_class)

		performance_class_count += 1
	
	print(f'Loaded {performance_class_count} custom performance classes.')




	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	## WAN PREFERENCES ##
	# We'll also count the number of rules to help the user know that it's working.
	rule_count = 0

	# For reference, the expected column order is
	# Protocol [0], Source [1], Src port [2], Destination [3], Dst port [4], Preferred uplink [5]

	# Append each WAN preference to loaded_wan_uplink_prefs
	for row in loaded_wan_prefs_worksheet.iter_rows(min_row=2):
		
		# Let's start with an empty rule dictionary to which we'll add the relevant parameters
		rule = {}

		# We know that there will always be a preferred uplink
		rule_preferred_uplink = row[5].value.lower()

		# The first column is Protocol
		rule_protocol = row[0].value.lower()
		
		# Source column is [1]
		rule_source = row[1].value

		# Destination column is [3]
		rule_destination = row[3].value.lower() 

		# Assemble the rule into a single Python object that uses the syntax that the corresponding API call expects
		if rule_protocol == 'any':
			# Since protocol is 'any' then src and dst ports are also 'any'
			rule_src_port = 'any'
			rule_dst_port = 'any'
			# Protocol is any, so leave out the port numbers
			rule_value = {
				'protocol': rule_protocol,
				'source': {
					'cidr': rule_source
				},
				'destination': {
					'cidr': rule_destination
				}
			}
		else:			
			# Since protocol is not 'any', we pass these as-is
			rule_src_port = row[2].value
			rule_dst_port = row[4].value
			# Rule isn't any, so we need the port numbers
			rule_value = {
				'protocol': rule_protocol,
				'source': {
					'port': rule_src_port,
					'cidr': rule_source
				},
				'destination': {
					'port': rule_dst_port,
					'cidr': rule_destination
				}
			}


		# Append the trafficFilters param to the rule
		rule['trafficFilters'] = [
			{
				'type': 'custom', # This worksheet doesn't have any Type column
				'value': rule_value
			}
		]

		# Append the preferredUplink param to the rule
		rule['preferredUplink'] = rule_preferred_uplink

		# Append the rule to the loaded_wan_uplink_prefs list
		loaded_wan_uplink_prefs.append(rule)

		rule_count += 1
	
	print(f'Loaded {rule_count} WAN uplink preferences.')

	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	## VPN PREFERENCES ##
	# For reference, the expected column order is
	# Type [0], Protocol or App ID [1], Source or App Name [2], Src port [3], Destination [4],
	# Dst port [5], Preferred uplink [6], Failover criterion [7], Performance class type [8],
	# Performance class name [9], Performance class ID [10]
	
	# We'll also count the number of rules to help the user know that it's working.
	rule_count = 0

	# Append each WAN preference to loaded_wan_uplink_prefs
	for row in loaded_vpn_prefs_worksheet.iter_rows(min_row=2):
	
		# Since the parameters can change depending on the various options, we'll start with an empty dictionary or list depending on parameter type and then add keys along the way to correspond to the relevant values.
		rule = {}
		rule_traffic_filters = {}
		
		# We know that there will always be a preferred uplink. We don't need any special logic to assign this one, so we'll keep it at the top.
		rule_preferred_uplink = row[6].value
		# Add it to the rule
		rule['preferredUplink'] = rule_preferred_uplink

		# The first column is Type, and the type will define the structure for other parameters.
		rule_type = row[0].value.lower() # Always lowercase.Chimpkennuggetss
		# If the rule type is application or applicationCategory then we're not concerned with destination, dst port or similar, and Protocol or App ID [1] will be 'id' not 'protocol', etc.
		if 'application' in rule_type:
			rule_application_id = row[1].value.lower() # Always lowercase
			rule_application_name = row[2].value # Leave it capitalized
			
			# Assign the rule value
			rule_value = {
				'id': rule_application_id, 
				'name': rule_application_name
			}

		else:
			# Assign the rule Protocol [1]
			rule_protocol = row[1].value.lower() # Always lowercase
			# Regardless of protocol, we need to assign Source [2]
			rule_source = row[2].value.lower() # Always lowercase
			# Regardless of protocol, we need to assign Destination [4]
			rule_destination = row[4].value.lower() # Always lowercase

			# Assign the rule ports, if appropriate
			if rule_protocol in ('any', 'icmp'):
				# Since protocol is 'any' or 'icmp' then we leave out src and dst ports 
				rule_value = {
					'protocol': rule_protocol, 
					'source': {
						'cidr': rule_source
					},
					'destination': {
						'cidr': rule_destination
					}
				}
			else:
				# Since protocol is not 'any', we pass these from the worksheet
				rule_src_port = row[3].value # Always lowercase
				rule_dst_port = row[5].value # Always lowercase
				rule_value = {
					'protocol': rule_protocol, 
					'source': {
						'port': rule_src_port,
						'cidr': rule_source
					},
					'destination': {
						'port': rule_dst_port,
						'cidr': rule_destination
					}
				}

		# Assemble the rule_traffic_filters parameter
		rule_traffic_filters['type'] = rule_type
		rule_traffic_filters['value'] = rule_value
		
		# Add it to the rule
		rule_traffic_filters_list = [rule_traffic_filters]
		rule['trafficFilters'] = rule_traffic_filters_list

		# Assign the optional failOverCriterion
		rule_failover_criterion = row[7].value # Leave it capitalized
		if rule_failover_criterion not in (string_constants['NOT_APPLICABLE'], ''):
			# Add it to the rule
			rule['failOverCriterion'] = rule_failover_criterion
		
		# Assign the optional performanceClass
		rule_performance_class_type = row[8].value
		rule_performance_class_name = row[9].value
		rule_performance_class_id = row[10].value
		if rule_performance_class_type not in (string_constants['NOT_APPLICABLE'], ''):
			# Add it to the rule
			rule['performanceClass'] = {}
			rule['performanceClass']['type'] = rule_performance_class_type
			
			# If the performance class type is custom, then we use customPerformanceClassId
			if rule_performance_class_type == 'custom':
				# Add it to the rule
				rule['performanceClass']['customPerformanceClassId'] = rule_performance_class_id
			# Otherwise, we use builtinPerformanceClassName
			else: 
				# Add it to the rule
				rule['performanceClass']['builtinPerformanceClassName'] = rule_performance_class_name

		# Append the rule to the loaded_vpn_uplink_prefs list
		loaded_vpn_uplink_prefs.append(rule)

		rule_count += 1
	
	print(f'Loaded {rule_count} VPN uplink preferences.')

	return(
		{
			'wanPrefs': loaded_wan_uplink_prefs, 
			'vpnPrefs': loaded_vpn_uplink_prefs, 
			'customPerformanceClasses': loaded_custom_performance_classes
		}
	)


In [ ]:
# We'll use the filename we specified at the top of the notebook.
# Load the workbook!
user_data['loaded_combined_uplink_prefs'] = load_uplink_prefs_workbook(WORKBOOK_FILENAME)

Let's take a look at those uplink preferences!

In [ ]:
printj(user_data['loaded_combined_uplink_prefs']['wanPrefs'])

# How might we look at the other components of the loaded backup?

## Restoring custom performance classes

Restoring custom performance classes can be tricky. IDs are unique, but a user might have changed the name or settings of a performance class after the last backup was taken, and that does not change the performance class's ID.

Given this scenario, and many other hypotheticals, we will simplify the restore operation with a straightforward and predictable behavior. It is designed to be most in-line with common expectations about what "restoring a backup" commonly means.

First we will check if the backup's classes are a perfect match to the currently configured ones. If so, there's no need to restore anything.

Otherwise, if the backup contains a performance class with the same ID as one that exists in the current Dashboard configuration, then we will overwrite that existing class with the settings (including name) from the backup.

Otherwise, if the backup contains a performance class with a different ID but the same name as one that exists in the current Dashboard configuration, then we will overwrite that existing class with the settings from the backup, and we will return the new/old IDs
in a list of key-value pairs so that we can update the corresponding `vpnUplinkPreference` to use this new performance class ID. If that happens, then when the uplink preferences are restored in a later function `update_vpn_prefs_performance_class_ids`, they will use the same performance class settings as were backed up, but the updated performance class ID.

Finally, if the backup contains a performance class that doesn't match any of the existing classes by name or ID, then we'll create it new, and return the new/old IDs as described above for a later function `update_vpn_prefs_performance_class_ids`.

In [ ]:
# A new function to compare current vs. loaded custom performance classes
# It will take as input the network ID and the list of the custom performance classes loaded from the backup
# If it has to update any VPN prefs' custom performance class IDs, it will do so, and return the new/old IDs
# in a list of key-value pairs. Otherwise, it will return None.
def restore_custom_performance_classes(*, listOfLoadedCustomPerformanceClasses, networkId):

	# Let's first make a list of the custom performance classes currently configured in the dashboard
	list_of_current_custom_performance_classes = dashboard.appliance.getNetworkApplianceTrafficShapingCustomPerformanceClasses(networkId=networkId)

	# Let's compare the currently configured classes with those from the backup. If they're the same, there's no need to restore anything.
	if list_of_current_custom_performance_classes == listOfLoadedCustomPerformanceClasses:
		print('The backed up custom performance classes matched the existing config.')
		return(None)

	# Otherwise, we will make several lists that we will use to compare the current config with the backup (loaded) config to determine what needs to be changed. We'll use the copy() method of the list object to make a new copy rather than creating a new reference to the original data.

	# We'll remove from this list as we find classes that match by either ID or name
	list_of_orphan_loaded_performance_classes = listOfLoadedCustomPerformanceClasses.copy()
	list_of_orphan_current_performance_classes = list_of_current_custom_performance_classes.copy()

	# We'll subtract from this list as we find classes that match by ID
	list_of_id_unmatched_current_performance_classes = list_of_current_custom_performance_classes.copy()

	# For those instances where we match by name, we'll store the new:old IDs in a new list
	list_of_id_updates = []

	# First let's check for current classes that match by ID. If they do, we'll update them with the loaded backup config.
	for loaded_performance_class in listOfLoadedCustomPerformanceClasses:

		# Let's look through each of the currently configured performance classes for that match
		for current_performance_class in list_of_current_custom_performance_classes:
			
			# Check if the IDs match up
			if loaded_performance_class['customPerformanceClassId'] == current_performance_class['customPerformanceClassId']:
				print(f"Matched {loaded_performance_class['customPerformanceClassId']} by ID! Restoring it from the backup.")
				# Restore that class from the loaded backup configuration
				dashboard.appliance.updateNetworkApplianceTrafficShapingCustomPerformanceClass(
					networkId=networkId, 
					customPerformanceClassId=current_performance_class['customPerformanceClassId'], 
					maxJitter=loaded_performance_class['maxJitter'],
					maxLatency=loaded_performance_class['maxLatency'],
					name=loaded_performance_class['name'],
					maxLossPercentage=loaded_performance_class['maxLossPercentage']
				)

				# Remove each from its respective orphan list
				list_of_orphan_loaded_performance_classes.remove(loaded_performance_class)
				list_of_orphan_current_performance_classes.remove(current_performance_class)
				
	# Let's next check the orphan lists for classes that match by name. If they do, we'll update them with the loaded backup config. 
	# If we find a match, we'll also add a reference object to the name-only match list tying the new ID to the respective one from
	# the loaded backup. If we find a match, we'll also remove it from both orphan lists.
	for orphan_loaded_performance_class in list_of_orphan_loaded_performance_classes:
		# Let's look through each of the currently configured performance classes for that match
		for orphan_current_performance_class in list_of_orphan_current_performance_classes:
			# Check if the names match up
			if orphan_loaded_performance_class['name'] == orphan_current_performance_class['name']:
				print(f"Matched custom performance class with ID {orphan_loaded_performance_class['customPerformanceClassId']} by name {orphan_loaded_performance_class['name']}! Restoring it from the backup.")
				# Restore that class from the loaded backup configuration
				dashboard.appliance.updateNetworkApplianceTrafficShapingCustomPerformanceClass(
					networkId=networkId, 
					customPerformanceClassId=orphan_current_performance_class['customPerformanceClassId'], 
					maxJitter=orphan_loaded_performance_class['maxJitter'],
					maxLatency=orphan_loaded_performance_class['maxLatency'],
					name=orphan_loaded_performance_class['name'],
					maxLossPercentage=orphan_loaded_performance_class['maxLossPercentage']
				)
				# Add it to the name-only matches list, list_of_name_matches
				list_of_id_updates.append(
					{
						'loaded_id': orphan_loaded_performance_class['customPerformanceClassId'], 
						'current_id': orphan_current_performance_class['customPerformanceClassId']
					}
				)
				# Remove each from its respective orphan list
				list_of_orphan_loaded_performance_classes.remove(orphan_loaded_performance_class)
				list_of_orphan_current_performance_classes.remove(orphan_current_performance_class)

	# If there are any orphans left, they have not matched by ID or name. Create them new.
	if len(list_of_orphan_loaded_performance_classes):
		print(f'{len(list_of_orphan_loaded_performance_classes)} new custom performance classes need to be created:')
		print(f'{list_of_orphan_loaded_performance_classes}\n')

		for orphan_loaded_performance_class in list_of_orphan_loaded_performance_classes:
			# Re-create the loaded class from the backup and get its new ID
			# We'll also add the old and new IDs to the reference object we've created for this purpose
			new_performance_class = dashboard.appliance.createNetworkApplianceTrafficShapingCustomPerformanceClass(
				networkId=networkId, 
				maxJitter=orphan_loaded_performance_class['maxJitter'],
				maxLatency=orphan_loaded_performance_class['maxLatency'],
				name=orphan_loaded_performance_class['name'],
				maxLossPercentage=orphan_loaded_performance_class['maxLossPercentage']
			)
			print(f'Created new custom performance class {new_performance_class} from the backup\'s {orphan_loaded_performance_class}.\n')
			
			# Add it to the name-only matches list, list_of_name_matches
			list_of_id_updates.append(
				{
					'loaded_id': orphan_loaded_performance_class['customPerformanceClassId'], 
					'current_id': new_performance_class['customPerformanceClassId']
				}
			)

	print('These IDs from the backup will need to be updated:')
	print(f'{list_of_id_updates}\n\n')
	# Return the list of updated IDs in key-value pairs for later processing
	return(list_of_id_updates)

## Restoring VPN uplink preferences

Now that the custom performance classes are restored, we can restore the VPN uplink preferences.

### Method to update the VPN prefs with updated performance class IDs

This nesting-doll of a function simply looks for `vpnUplinkPreferences` that use custom performance classes, and updates those IDs to match any corresponding new ones created, such as when a backed up performance class was deleted after the backup.

In [ ]:
def update_vpn_prefs_performance_class_ids(*, loaded_vpn_prefs, performance_class_id_updates):
    vpn_prefs_updates = 0
    # For each update in the ID updates list
    for update in performance_class_id_updates:
        # For each rule in loaded_vpn_prefs
        for rule in loaded_vpn_prefs:
            # If the rule's performance class type is set
            if 'performanceClass' in rule.keys():
                # If the rule's performance class type is custom
                if rule['performanceClass']['type'] == 'custom':
                    # And if the rule's customPerformanceClassId matches one from our ID updates list
                    if rule['performanceClass']['customPerformanceClassId'] == update['loaded_id']:
                        # Then we update it with the new ID
                        rule['performanceClass']['customPerformanceClassId'] = update['current_id']
                        vpn_prefs_updates += 1
    
    return(vpn_prefs_updates)

### Method to restore the VPN preferences to Dashboard

Specify the `networkId` and provide the VPN uplink preferences as a list. This method is documented [here](https://developer.cisco.com/meraki/api-v1/#!update-network-appliance-traffic-shaping-uplink-selection).

> NB: Setting a variable `response` equal to the SDK method is a common practice, because the SDK method will return the API's HTTP response. That information is useful to confirm that the operation was successful, but it is not strictly required.

In [ ]:
# A new function to push VPN preferences
def restore_vpn_prefs(backup_vpn_prefs_list):
	current_vpn_prefs_list = dashboard.appliance.getNetworkApplianceTrafficShapingUplinkSelection(
		networkId=user_choices['network_choice'].id
	)['vpnTrafficUplinkPreferences']
	
	if current_vpn_prefs_list == backup_vpn_prefs_list: 
		print(f'The current VPN prefs list matches the backup. No VPN prefs changed.')
		return(None)

	else:
		response = dashboard.appliance.updateNetworkApplianceTrafficShapingUplinkSelection(
			networkId=user_choices['network_choice'].id, 
			vpnTrafficUplinkPreferences=backup_vpn_prefs_list
		)
		print(f'The VPN prefs list was restored.')
		return(response)

### Method to restore the WAN preferences to Dashboard

Specify the `networkId` and provide the WAN uplink preferences as a list. This method is documented [here](https://developer.cisco.com/meraki/api-v1/#!update-network-appliance-traffic-shaping-uplink-selection).

> NB: Notice that this relies on the same SDK method as `restore_vpn_prefs()` above. Here we've split the restore into two functions to demonstrate that you can push only specific keyword arguments when the other keyword arguments are optional. Since it's the same method, we could consolidate this function, `restore_wan_prefs()`, and `restore_vpn_prefs()`, into a single function by passing both keyword arguments `vpnTrafficUplinkPreferences` and `wanTrafficUplinkPreferences` at the same time. This would then increase the amount of work accomplished by a single API call. We recommend following a best practice of accomplishing as much as possible with as few calls as possible, when appropriate.

In [ ]:
# A new function to push WAN preferences
def restore_wan_prefs(backup_wan_prefs_list):
	current_wan_prefs_list = dashboard.appliance.getNetworkApplianceTrafficShapingUplinkSelection(
		networkId=user_choices['network_choice'].id
	)['wanTrafficUplinkPreferences']

	if current_wan_prefs_list == backup_wan_prefs_list:
		print(f'The current WAN prefs list matches the backup. No WAN prefs changed.')
		return(None)

	else: 
		response = dashboard.appliance.updateNetworkApplianceTrafficShapingUplinkSelection(
			networkId=user_choices['network_choice'].id, 
			wanTrafficUplinkPreferences=backup_wan_prefs_list
			)
		print(f'The WAN prefs list was restored.')
		return(response)

## Wrapping up!

We've now built functions to handle the discrete tasks required to restore the configuration for the three items:

* WAN uplink preferences
* VPN uplink preferences
* Custom performance classes

We had to translate the Excel workbook into a Python object that was structured according to the API specifications.

We found that some extra logic was required to properly restore custom performance classes and VPN uplink preferences that use them, and wrote custom functions to handle it. 

> NB: We handled this one way of potentially many. Can you think of any other ways you might handle the problem of missing custom performance class IDs, or overlapping names or IDs?

However, we haven't actually called any of these functions, so the restore hasn't happened! To actually call those functions, we'll run them in the next cell.

### Restore the backup!

In [ ]:
# Restore the custom performance classes
user_data['updated_performance_class_ids'] = restore_custom_performance_classes(
	listOfLoadedCustomPerformanceClasses=user_data['loaded_combined_uplink_prefs']['customPerformanceClasses'], 
	networkId=user_choices['network_choice'].id
	)

# Update the custom perfromance class IDs
if user_data['updated_performance_class_ids']:
    update_vpn_prefs_performance_class_ids(
        loaded_vpn_prefs=user_data['loaded_combined_uplink_prefs']['vpnPrefs'],
        performance_class_id_updates=user_data['updated_performance_class_ids']
    )

# Restore the VPN prefs
user_data['restored_vpn_prefs'] = restore_vpn_prefs(
    user_data['loaded_combined_uplink_prefs']['vpnPrefs']
)

# Restore the WAN prefs
user_data['restored_wan_prefs'] = restore_wan_prefs(
    user_data['loaded_combined_uplink_prefs']['wanPrefs']
)

# Final thoughts

And we're done! Hopefully this was a useful deep dive into Python programming and interacting with the Meraki SDK and Excel workbooks. We tackled a problem that is tough to solve in the Dashboard GUI and showed how it can be done very quickly via API and the Python SDK.

Here we used Excel workbooks, but you can imagine that there are all types of data structures that might be used instead of Excel workbooks, e.g. CSVs, plain text, YAML, XML, LibreOffice files or others, and with the right code you can use those instead of Excel.

## Further learning

Consider whether there are any "gotchas" for which we didn't prepare in this code demo. For example, what if you already have the maximum number of custom performance classes defined in the Dashboard--how might that interfere with your ability to restore the backup using this code? What might you add or change to handle that possibility?

[Meraki Interactive API Docs](https://developer.cisco.com/meraki/api-v1/#!overview): The official (and interactive!) Meraki API and SDK documentation repository on DevNet.